In [4]:
import pandas as pd
import numpy as np

In [5]:
train = pd.read_csv('./Assets/train_users_2.csv/train_users_2.csv')
test = pd.read_csv('./Assets/test_users.csv/test_users.csv')
print train.shape
print test.shape

(213451, 16)
(62096, 15)


In [6]:
test.drop(39024,inplace=True)

### Equalize features in dataframes to be combined

In [7]:
[i for i in train.columns.values if i not in test.columns.values]

['country_destination']

In [8]:
y_train = train['country_destination']

In [9]:
# train.drop([i for i in train.columns.values if i not in test.columns.values], inplace=True, axis = 1)

In [10]:
combined = train.append(test)

In [9]:
print combined.shape
combined.info()

(275546, 16)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 275546 entries, 0 to 62095
Data columns (total 16 columns):
affiliate_channel          275546 non-null object
affiliate_provider         275546 non-null object
age                        158681 non-null float64
country_destination        213451 non-null object
date_account_created       275546 non-null object
date_first_booking         88908 non-null object
first_affiliate_tracked    269461 non-null object
first_browser              275546 non-null object
first_device_type          275546 non-null object
gender                     275546 non-null object
id                         275546 non-null object
language                   275546 non-null object
signup_app                 275546 non-null object
signup_flow                275546 non-null int64
signup_method              275546 non-null object
timestamp_first_active     275546 non-null int64
dtypes: float64(1), int64(2), object(13)
memory usage: 35.7+ MB


In [10]:
brow_cnt = zip(combined['first_browser'].value_counts().index, combined['first_browser'].value_counts().values)
brow_other = [x[0] for x in brow_cnt if x[1]<150]
combined.first_browser = combined.first_browser.apply(lambda x: 'Other' if x in brow_other else x)

In [11]:
### age_x_test and age_x_train represent the set to give me ages

### with ids they will be merged later back into train and test originals

In [12]:
age_x_test = combined[combined.age.isnull()] 
print 'x_test :', age_x_test.shape
age_x_train = combined[combined.age.notnull()] 
print 'x_train :', age_x_train.shape
age_target = age_x_train.age
print 'age target :', age_target.shape


x_test : (116865, 16)
x_train : (158681, 16)
age target : (158681L,)


In [13]:
# pd.to_datetime(age_x_train.timestamp_first_active.map(lambda x: str(x)[0:8]))

In [1]:
features = ['gender', 'signup_method', 'signup_app', 'first_device_type','affiliate_provider', 'first_browser']
for f in features:
    print f, len(age_x_train[f].unique())
    print age_x_train[f].value_counts()
    print '------------------------------'

gender

NameError: name 'age_x_train' is not defined

In [28]:
for f in features:
    print f, len(age_x_test[f].unique())
    print age_x_test[f].value_counts()
    print '------------------------------'

gender 4
-unknown-    107242
FEMALE         5617
MALE           3951
OTHER            55
Name: gender, dtype: int64
------------------------------
signup_method 4
basic       112617
facebook      2312
google        1920
weibo           16
Name: signup_method, dtype: int64
------------------------------
language 24
en    112899
zh      1275
ko       538
fr       474
de       446
es       368
it       182
ru       157
ja       139
pt        91
sv        66
nl        61
da        34
tr        31
no        25
cs        24
pl        23
hu         9
th         7
fi         5
id         5
el         4
ca         1
is         1
Name: language, dtype: int64
------------------------------
signup_app 4
Web        87094
iOS        17249
Moweb       6339
Android     6183
Name: signup_app, dtype: int64
------------------------------
first_device_type 9
Mac Desktop           37290
Windows Desktop       36744
iPhone                20154
iPad                   8543
Other/Unknown          6917
Android P

In [16]:
train_ = age_x_train[features]
X = pd.get_dummies(train_)
X.head(1)

,gender_-unknown-,gender_FEMALE,gender_MALE,gender_OTHER,signup_method_basic,signup_method_facebook,signup_method_google,signup_method_weibo,language_ca,language_cs,...,first_browser_Android Browser,first_browser_Chrome,first_browser_Chrome Mobile,first_browser_Firefox,first_browser_IE,first_browser_Mobile Safari,first_browser_Opera,first_browser_Other,first_browser_Safari,first_browser_Silk
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
print X.shape, age_target.shape

(158681, 76) (158681L,)


In [18]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X, age_target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [19]:
test_ = age_x_test[features]
X_test = pd.get_dummies(test_)

In [20]:
print X.shape
print X_test.shape

(158681, 76)
(116865, 75)


In [21]:
# X.columns.values
[i for i in X.columns.values if i not in X_test.columns.values]

['language_hr']

In [22]:
[i for i in X_test.columns.values if i not in X.columns.values]

[]

In [23]:
pd.Series(model.predict(X_test)).map(lambda x: x if x < 75 else 75).map(lambda x: x if x > 18 else 18).unique()

ValueError: Number of features of the model must match the input. Model n_features is 76 and input n_features is 75 

In [ ]:
age_x_test

In [ ]:
age_x_test.loc[:,'age'] = pd.Series(model.predict(allxtest)).map(lambda x: x if x < 75 else 75).map(lambda x: x if x > 18 else 18)

In [ ]:
age_x_test.age

In [ ]:
a_train = pd.merge( left=train, right=age_x_test , on= 'id', how = 'inner' )

In [ ]:
a1_train = pd.merge( left=train, right=age_x_train, on= 'id', how = 'inner' )

In [ ]:
train_merge = pd.concat([a_train, a1_train], axis=0)

In [ ]:
train_merge.drop([i for i in train_merge.columns if '_x' in i], axis=1, inplace = True)

In [ ]:
train_merge.columns = train_merge.columns.str.replace('_y', '')
train_merge.columns

In [ ]:
train_merge.info()

In [ ]:
print train.shape 
print train_merge.shape

In [ ]:
train_merge.to_pickle('train_age')